<a href="https://colab.research.google.com/github/dyjdlopez/intro_2_quantum/blob/main/cuda-q/01-introduction-to-quantum-programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Introduction to CUDA-Q and Fundamental Quantum Computing
$_{\text{Made by: @dyjdlopez}}$


In this notebook we will look at initial steps in programming quantum algorithms with CUDA-Q.

CUDA-Q is **NVIDIA’s platform** for accelerating quantum computing using GPUs. It enables efficient execution of quantum algorithms by integrating classical and quantum processing. Designed for both beginners and experts, CUDA-Q supports **quantum simulations, optimization, and AI applications**, allowing users to explore quantum computing without direct access to quantum hardware.
For more details, visit the **[CUDA-Q official website](https://developer.nvidia.com/cuda-quantum)**.

![image](https://www.nvidia.com/content/nvidiaGDC/us/en_US/solutions/quantum-computing/_jcr_content/root/responsivegrid/nv_container_1856196339/nv_container/nv_image.coreimg.svg/1731940253298/cuda-quantum-diagram.svg)

 Although CUDA-Q is not the  initial choice for quantum algorithim programming (unlike Qiskit), its practical use with GPU acceleration allows it to become a good choice for near real-time simulation. Since the flow of the course focuses on the practical applications of quantum computing, it is essential to employ its deployability and interoperability with GPU hardware.

 ![image](https://www.nvidia.com/content/nvidiaGDC/us/en_US/solutions/quantum-computing/_jcr_content/root/responsivegrid/nv_container_1856196339/nv_container_1543699179/nv_image.coreimg.svg/1731940253076/nvidia-quantum-cloud-diagram.svg)


In [ ]:
!pip install cudaq==0.9.1

In [ ]:
import cudaq
import math

# 1.1 Qubits and Quantum Memory  

In **quantum computing**, a **qubit** (quantum bit) is the fundamental unit of information, capable of existing in **superposition** (both 0 and 1 simultaneously) and **entanglement** (correlated with other qubits).  

In **CUDA-Q** , qubits are stored in **[quantum vectors (`qvector`)](https://nvidia.github.io/cuda-quantum/api_docs/functions/qvector.html)**, which act as **quantum memory** for running quantum circuits. CUDA-Q allows users to **allocate qubits, apply quantum gates, and measure states** efficiently using GPU acceleration.


In [ ]:
qubit = cudaq.qvector(1) # single qubit
qubits = cudaq.qvector(2) # quantum memory consisting of 2 qubits

In [ ]:
qubit.size() # checks the size of a quantum memory

1

# 1.2 Quantum Kernels

However, it might not be easy to diagnose the statevector of the quantum memory as is. In CUDA-Q you must create quantum circuit or an ensemble to draw and evaluate quantum states.

A **[quantum kernel](https://nvidia.github.io/cuda-quantum/api_docs/python/cudaq.kernel.html)** in CUDA-Q is a function marked with the `@cudaq.kernel` decorator, which indicates that the function contains quantum operations. These operations are executed on quantum hardware or simulators with GPU acceleration.

In [ ]:
@cudaq.kernel
def quantum_circuit():
    qubits = cudaq.qvector(3)
print(cudaq.get_state(quantum_circuit))

SV: [(1,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0), (0,0)]



# 1.3 Quantum Gates

However, there's nothing exciting with our qubits yet. Ot make it more exciting we need to apply quantum gates!

## 1.3.1 Single-qubit Gates

**Single-qubit quantum gates** are operations that affect the state of a single qubit. These gates are fundamental in creating superposition, entanglement, or applying rotations in quantum circuits. For more information on single-qubit **[gates](https://nvidia.github.io/cuda-quantum/api_docs/python/cudaq.gates.html)** in CUDA-Q, refer to the documentation.

**Common Single-Qubit Gates:**

1. **Hadamard Gate (`h`)**  
   The Hadamard gate creates a superposition of the $|0⟩$ and $|1⟩$ states. The matrix representation of the Hadamard gate is:

   $$
   H = \frac{1}{\sqrt{2}} \begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix}
   $$

   This maps the |0⟩ state to $|+⟩ = \frac{|0⟩ + |1⟩}{\sqrt{2}}$ and the $|1⟩$ state to $|-⟩=\frac{|0⟩ - |1⟩}{\sqrt{2}}$.

2. **Pauli Gates (X, Y, Z)**  
   - **X Gate (NOT gate)**: Flips the state of the qubit, represented as:
  $$X= \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix} $$
   - **Y Gate**: Applies a rotation along the Y axis:

  $$
     Y = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix}
  $$

   - **Z Gate**: Applies a rotation along the Z axis:

  $$
     Z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}
  $$

3. **Phase Gates (S, T)**  
   - **S Gate**: Rotates the qubit by $\frac{\pi}{2}$:
  $$
     S = \begin{pmatrix} 1 & 0 \\ 0 & i \end{pmatrix}
  $$

   - **T Gate**: Rotates the qubit by $\frac{\pi}{4}$:

  $$
     T = \begin{pmatrix} 1 & 0 \\ 0 & e^{i\pi/4} \end{pmatrix}
  $$

In [ ]:
# Try to experiment on this circuit to change the gates
@cudaq.kernel
def quantum_circuit():
    qubits = cudaq.qvector(1)  # Declare a quantum vector with 1 qubit
    h(qubits[0])               # Apply Hadamard gate (creates superposition)
    x(qubits[0])               # Apply X gate (flips the qubit)

print(cudaq.draw(quantum_circuit)) ## make a simple visualization of the circuit.
print(cudaq.get_state(quantum_circuit))

     ╭───╮╭───╮
q0 : ┤ h ├┤ x ├
     ╰───╯╰───╯

SV: [(0.707107,0), (0.707107,0)]



## 1.3.2 Measurement Gates

In quantum computing, **measurement** is the process of determining the state of a qubit. Unlike classical bits, quantum bits (qubits) exist in a superposition of states, and measuring a qubit forces it into one of its basis states ($|0⟩$ or $|1⟩$). This process is crucial for extracting information from a quantum system.

In **CUDA-Q**, measurement is done using the **`mz`** (measure) gate. The `mz` gate measures the state of a quantum vector and collapses it into one of the basis states, either $|0⟩$ or $|1⟩$. After measurement, the result can be used in classical calculations.

Measurement is an essential part of quantum algorithms as it provides the classical output after quantum operations.



In [ ]:
@cudaq.kernel
def quantum_circuit():
    qubit = cudaq.qvector(1)  # Declare a quantum vector with 1 qubit
    h(qubit[0])               # Apply Hadamard gate (creates superposition)
    mz(qubit)                 # Measure the qubit and store the result


# Execute the quantum circuit and get measurement results
print(cudaq.draw(quantum_circuit))
result = cudaq.sample(quantum_circuit)
print(result)
print(cudaq.get_state(quantum_circuit))

     ╭───╮
q0 : ┤ h ├
     ╰───╯

{ 0:513 1:487 }

SV: [(0,0), (1,0)]



## 1.3.3 Multi-Qubit Operations in CUDA-Q

Multi-qubit operations are fundamental for creating entanglement and enabling quantum algorithms. In **CUDA-Q**, you can perform multi-qubit operations on quantum vectors, manipulating multiple qubits simultaneously.

**Key Multi-Qubit Operations:**

1. **CNOT Gate (Controlled-NOT)**  
   The **CNOT gate** creates entanglement between qubits. It applies a **NOT** operation to the target qubit only if the control qubit is in the |1⟩ state. The matrix representation is:

   $$
   \text{CNOT} = \begin{pmatrix}
   1 & 0 & 0 & 0 \\
   0 & 1 & 0 & 0 \\
   0 & 0 & 0 & 1 \\
   0 & 0 & 1 & 0
   \end{pmatrix}
   $$

2. **SWAP Gate**  
   The **SWAP gate** exchanges the states of two qubits. It is represented as:

   $$
   \text{SWAP} = \begin{pmatrix}
   1 & 0 & 0 & 0 \\
   0 & 0 & 1 & 0 \\
   0 & 1 & 0 & 0 \\
   0 & 0 & 0 & 1
   \end{pmatrix}
   $$

3. **Toffoli Gate (Controlled-Controlled-NOT, CCNOT)**  
   The **Toffoli gate** (also known as the **CCNOT gate**) flips the target qubit if both control qubits are in the |1⟩ state. The matrix for a 3-qubit Toffoli gate is:

   $$
   \text{CCNOT} = \begin{pmatrix}
   1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
   0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
   0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
   0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
   0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
   0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
   0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
   0 & 0 & 0 & 0 & 0 & 0 & 1 & 0
   \end{pmatrix}
   $$

In [ ]:
@cudaq.kernel
def quantum_circuit():
    qubits = cudaq.qvector(2)   # Declare a quantum vector with 2 qubits
    h(qubits[0])                # Apply Hadamard gate on qubit 0 (creates superposition)
    cx(qubits[0], qubits[1])    # Apply CNOT gate (entangles qubits 0 and 1)
    # mz(qubits)


print(cudaq.draw(quantum_circuit))
print(cudaq.get_state(quantum_circuit))

# To observe the probabilities, uncomment the Mz gate
# result = cudaq.sample(quantum_circuit)
# print(result)

     ╭───╮     
q0 : ┤ h ├──●──
     ╰───╯╭─┴─╮
q1 : ─────┤ x ├
          ╰───╯

SV: [(0.707107,0), (0,0), (0,0), (0.707107,0)]



In [ ]:
## Alternative way for creating a CNOT operation
@cudaq.kernel
def quantum_circuit():
    qubits = cudaq.qvector(2)
    h(qubits[0])
    x.ctrl(qubits[0], qubits[1])
    mz(qubits)

# Execute the quantum circuit and get measurement results
print(cudaq.draw(quantum_circuit))
result = cudaq.sample(quantum_circuit)
print(result)

     ╭───╮     
q0 : ┤ h ├──●──
     ╰───╯╭─┴─╮
q1 : ─────┤ x ├
          ╰───╯

{ 00:490 11:510 }



In [ ]:
@cudaq.kernel
def quantum_circuit():
    qubits = cudaq.qvector(2)
    x(qubits[0])
    swap(qubits[0], qubits[1])
    mz(qubits)

# Execute the quantum circuit and get measurement results
print(cudaq.draw(quantum_circuit))
result = cudaq.sample(quantum_circuit)
print(result)

     ╭───╮   
q0 : ┤ x ├─╳─
     ╰───╯ │ 
q1 : ──────╳─
             

{ 01:1000 }



### Exercise 1
Create a Toffoli Gate for a 3-qubit memory in CUDA-Q. You may read the documentation on [Quantum Operations](https://nvidia.github.io/cuda-quantum/latest/api/default_ops.html#) further for clues.

In [ ]:
@cudaq.kernel
def act1_qc():
  num_qubits = 0 # Edit this part
  qubits = cudaq.qvector(num_qubits)
  ## Add the code for the Toffoli gate after this line

  ## Do not edit beyond this point
  mz(qubits)

print(cudaq.draw(act1_qc))
result = cudaq.sample(act1_qc)
print(result)

Expected output:

```
          
q0 : ──●──
       │  
q1 : ──●──
     ╭─┴─╮
q2 : ┤ x ├
     ╰───╯
{ 000:1000 }
```






# 1.4 Parametric Gates
Parametric gates are quantum gates that take a parameter, typically an angle, which determines the operation applied to a qubit. These gates are essential for performing rotations and are often used in quantum algorithms to manipulate qubit states in a controlled manner.

## 1.4.1 Rotation Gates


1. **RZ Gate (Rotation around the Z-axis)**  
   The **RZ gate** performs a rotation around the Z-axis by an angle $\theta$. The matrix representation is:

   $$
   R_Z(\theta) = \begin{pmatrix}
   e^{-i\theta/2} & 0 \\
   0 & e^{i\theta/2}
   \end{pmatrix}
   $$

   This gate shifts the phase of the qubit.

2. **RX Gate (Rotation around the X-axis)**  
   The **RX gate** rotates the qubit around the X-axis by an angle $theta$. The matrix for this gate is:

   $$
   R_X(\theta) = \begin{pmatrix}
   \cos(\theta/2) & -i\sin(\theta/2) \\
   -i\sin(\theta/2) & \cos(\theta/2)
   \end{pmatrix}
   $$

3. **RY Gate (Rotation around the Y-axis)**  
   The **RY gate** rotates the qubit around the Y-axis by an angle $\theta$. Its matrix representation is:

   $$
   R_Y(\theta) = \begin{pmatrix}
   \cos(\theta/2) & -\sin(\theta/2) \\
   \sin(\theta/2) & \cos(\theta/2)
   \end{pmatrix}
   $$

In [ ]:
@cudaq.kernel
def quantum_circuit():
    qubits = cudaq.qvector(1)
    rz(math.pi/4, qubits[0])
    rx(math.pi/3, qubits[0])
    ry(math.pi/6, qubits[0])

print(cudaq.draw(quantum_circuit))
print(cudaq.get_state(quantum_circuit))

     ╭────────────╮╭───────────╮╭────────────╮
q0 : ┤ rz(0.7854) ├┤ rx(1.047) ├┤ ry(0.5236) ├
     ╰────────────╯╰───────────╯╰────────────╯

SV: [(0.822363,-0.200562), (0.02226,-0.531976)]



### Exercise 2
Create a parametrized circuit that takes in three variables `theta_1`, `theta_2`, and `theta_3`. Follow the circuit design below:


```
     ╭────────────╮╭───────────╮╭────────────╮
q0 : ┤rx(theta_1) ├┤ry(theta_2)├┤ rz(theta_3)├
     ╰────────────╯╰───────────╯╰────────────╯
```

Note that the variable names should not be reflected in printing the circuit drawing, they should show numerical values.

## 1.4.2 The U3 Gate


The **U3 gate** is a universal single-qubit gate that allows for rotations on a qubit with three parameters: **$\theta$**, **$\phi$**, and **$\lambda$**. These parameters represent Euler angles and define the rotation of a qubit on the Bloch sphere. The matrix representation of the U3 gate is:

$$
U_3(\theta, \phi, \lambda) = \begin{pmatrix}
\cos(\theta/2) & -e^{i\lambda} \sin(\theta/2) \\
e^{i\phi} \sin(\theta/2) & e^{i(\phi + \lambda)} \cos(\theta/2)
\end{pmatrix}
$$

Where:
- **$\theta$** is the rotation angle.
- **$\phi$** and **$\lambda$** are phase parameters that influence the qubit's components.

This gate is universal, meaning it can simulate any single-qubit operation by selecting the appropriate values for **$\theta$**, **$\phi$**, and **$\lambda$**. It can reproduce gates like **Hadamard**, **Pauli-X**, **Pauli-Y**, **Pauli-Z**, and others.


In [ ]:
@cudaq.kernel
def quantum_circuit(theta: float, phi:float, lmbda:float):
  qubits = cudaq.qvector(1)
  u3(theta, phi, lmbda, qubits[0])

th, ph, lmb = math.pi/2, math.pi, math.pi/4
print(cudaq.draw(quantum_circuit, th, ph, lmb))
state = cudaq.get_state(quantum_circuit, th, ph, lmb)
print(state)

     ╭────────────────────────╮
q0 : ┤ u3(1.571,3.142,0.7854) ├
     ╰────────────────────────╯

SV: [(0.707107,0), (-0.707107,-6.18172e-08)]



### Exercise 3
What values of $\theta$, $\phi$, and $\lambda$ that would make the U3 gate have equal statevector values as the $H$, $X$, $Y$, $Z$, and $T$ gates?

Use whichever method that you could use. Provide the mathematical proof and solution. (You are expected to provide 5 sets of $\theta$, $\phi$, and $\lambda$ values corresponding to the gates.)

---

---
$$_{\text{END OF FILE}}$$
$$_{\text{D.J.D. Lopez | © 2025}}$$